In [4]:
import torch
import torch.nn.functional as F
from urllib.request import urlopen
from PIL import Image
from open_clip import create_model_from_pretrained, get_tokenizer 

model, preprocess = create_model_from_pretrained('hf-hub:apple/DFN5B-CLIP-ViT-H-14')
tokenizer = get_tokenizer('ViT-H-14')

image = Image.open("./Scene/0/0.jpg")
image = preprocess(image).unsqueeze(0)

labels_list = ['Buildings', 'Forests', 'Glacier', 'Mountains', 'Sea', 'Street']
text = tokenizer(labels_list, context_length=model.context_length)

In [5]:
with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features = F.normalize(image_features, dim=-1)
    text_features = F.normalize(text_features, dim=-1)

    text_probs = torch.sigmoid(image_features @ text_features.T * model.logit_scale.exp())

zipped_list = list(zip(labels_list, [round(p.item(), 3) for p in text_probs[0]]))
print("Label probabilities: ", zipped_list)

Label probabilities:  [('Buildings', 0.913), ('Forests', 0.73), ('Glacier', 0.866), ('Mountains', 0.636), ('Sea', 0.716), ('Street', 0.967)]


In [3]:
from datetime import datetime
import pandas as pd
import os

In [ ]:
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f'submission_{current_time}.csv'
print(file_name)

img_nums = 8100
submission = dict({'id_idx': [], 'label': []})

for idx in range(img_nums):
    image = Image.open(f"./Scene/0/{idx}.jpg")
    image = preprocess(image).unsqueeze(0)

    with torch.no_grad(), torch.cuda.amp.autocast():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
        image_features = F.normalize(image_features, dim=-1)
        text_features = F.normalize(text_features, dim=-1)    
        text_probs = torch.sigmoid(image_features @ text_features.T * model.logit_scale.exp())

    submission['label'] += [int(text_probs.argmax().cpu())]
    
    if (idx+1)%100 == 0:
        submission['id_idx'] = list(range(len(submission['label'])))
        pd.DataFrame(submission).to_csv(os.path.join("./submissions/", file_name), index=False)
        print((idx+1)//100, end=" ")

submission_20240828_165031.csv
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 